In [ ]:
# periodic time series

#  time series modeling process

# 1. stationary test
#    trend model:
#         Aggregation – taking average for a time period like monthly/weekly averages
#         Smoothing – taking rolling averages
#         Polynomial Fitting – fit a regression model
#    seasonality model:
#         differencing
#         model fitting
# 2. stationalize time series
#       detrend
#       remove seasonality
# 3. model the stationary part of time series        

#    seasonal-ARIMA 

# 4. perform prediction 


# TO DO
# plain time series prediction
# periodic time series prediction

# https://dandelion.eu/datamine/open-big-data/


In [ ]:
# http://www.johnwittenauer.net/a-simple-time-series-analysis-of-the-sp-500-index/
 

In [ ]:
# TO DO: 

# Statistics programming 

# book:

# Probabilistic Programming and Bayesian Methods for Hackers 
# http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Prologue/Prologue.ipynb

In [ ]:
# Examples:

# stock prediction
# https://bitbucket.org/joexdobs/ml-classifier-gesture-recognition/wiki/stock-example/
# predict-future-stock-price-using-machine-learning.md

# statsmodels:
#  http://statsmodels.sourceforge.net/stable/index.html

# http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/

# http://nbviewer.jupyter.org/gist/ChadFulton/5127108f4c7025ed2648

# https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/

# http://nbviewer.jupyter.org/github/jakevdp/SeattleBike/blob/master/SeattleCycling.ipynb

# book
# http://shelfjoy.com/shelfjoy/17-essential-machine-learning-books-suggested-by-michael-i-jordan-from-berkeley

In [4]:
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')

from utils_libs import *
from utils_data_prep import *

from scipy.stats import lognorm
from scipy.stats import norm
from scipy.stats import chisqprob

from numpy import prod
import seaborn as sns

# statiscal models
import statsmodels as sm
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.api import VAR, DynamicVAR

from statsmodels.stats import diagnostic


/Users/anino/miniconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
# --- Load pre-processed order book data ---

all_dta_minu = np.load("../dataset/bitcoin/dta_minu.dat")
all_loc_hour = np.load("../dataset/bitcoin/loc_hour.dat")
print len(all_dta_minu), len(all_loc_hour)

# --- Load order book data files ---

# all_dta_minu,all_loc_hour = load_raw_order_book_files('../dataset/bitcoin/order_book/*.csv', True)

539157 10641


In [3]:
# --- calculate price, return and volatility ---

price_minu, req_minu = cal_price_req_minu(all_dta_minu)

print np.shape(price_minu), np.shape(req_minu)

pvol_hour = cal_price_volatility_hour( all_loc_hour, price_minu )
return_minu, rvol_hour = cal_return_volatility_hour( all_loc_hour, price_minu, 'per' )

print len(price_minu),len(return_minu), len(pvol_hour), len(rvol_hour)

(539157,) (539157, 2)
539157 528516 10641 10641


In [6]:
# --- distributional features ---

# analytical posterior: sampling by enumerating and calculating density
# approximate posterior: sampling via MCMC
    
# pymc?
def poterior_sample_norm_2d(x, n_samples):
    return 1
    
def poterior_sample_log_norm_2d(x, n_samples):
    return 1
    
def map_log_norm_2d(x):
    
    if len(x) == 0:
        return [0.0, 0.0], [0.0, 0.0, 0.0]
    
    elif len(x) == 1:
        return [ x[0][0], x[0][1] ], [0.0, 0.0, 0.0]
    
    else:
        
        tmpx = [ [i[0]+1e-5, i[1]+1e-5] for i in x]
        logx = log(tmpx)
        
        post_log_mu, post_log_cov = map_norm_2d(logx)
        
        post_mu = [ exp(post_log_mu[i] + post_log_cov[i]/2.0) for i in range(2) ]
        
#         post_cov = [ [0.0, 0.0] for i in range(2) ]
        
#         for i in range(2):
#             for j in range(2):
#                 tmp = post_log_cov[2] if i!=j else post_log_cov[i]  
#                 post_cov[i][j] = exp( post_log_mu[i]+post_log_mu[j]+0.5*(post_log_cov[i]+post_log_cov[j]) )*\
#                 ( exp(tmp)-1.0 )
                
        var0 = exp( post_log_mu[0]+post_log_mu[0]+0.5*(post_log_cov[0]+post_log_cov[0]) )*\
        ( exp(post_log_cov[0])-1.0 )
        var1 = exp( post_log_mu[1]+post_log_mu[1]+0.5*(post_log_cov[1]+post_log_cov[1]) )*\
        ( exp(post_log_cov[1])-1.0 )
        cov = exp( post_log_mu[0]+post_log_mu[1]+0.5*(post_log_cov[0]+post_log_cov[1]) )*\
        ( exp(post_log_cov[2])-1.0 )
        
        return list(post_mu), [var0, var1, cov], list(post_mu), \
               [post_log_cov[0][0], post_log_cov[1][1], post_log_cov[0][1]] 

def map_norm_2d( x ):
    
    if len(x) == 0:
        return [0.0, 0.0], [0.0, 0.0, 0.0]
    
    elif len(x) == 1:
        return [ x[0][0], x[0][1] ], [0.0, 0.0, 0.0]
    
    else:
        mle_mu  = np.mean(x, axis=0) 
        mle_cov = np.cov(x, rowvar=0)
    
        m_0 = mle_mu
        k0  = 0.01
        v0 = 2.0 + 2.0
        S_0 = np.diag(np.diag(mle_cov))*1.0/len(x)
    
        x_ba = mle_mu
    
        #S = np.zeros((2, 2))
        #for i in x:
        #    S = np.add(S, np.outer(i, i))
            
        S = np.matmul( np.asmatrix(x).transpose() , np.asmatrix(x) )
        
        N = len(x)
        m_N = k0*1.0/(k0+N)*m_0 + N*1.0/(k0+N)*x_ba 
    
        vN = v0 + N    
        kN = k0 + N
        
        S_N = S_0 + S + k0*np.outer(m_0, m_0) - kN*np.outer(m_N, m_N)
        
        cov_mode = S_N*1.0/(vN+2.0+2.0)
        
        return list(m_N), [ cov_mode.item((0, 0)), cov_mode.item((1, 1)), cov_mode.item((1, 0)) ] 
    
def mle_norm_2d( x ):
    
    if len(x) == 0:
        return [0.0, 0.0], [0.0, 0.0]
    elif len(x) == 1:
        return [x[0][0], x[0][1]], [0.0, 0.0]
    else:
        tmp = np.cov(x, rowvar=0)
        return list(np.mean(x, axis=0)), [tmp[0][0], tmp[1][1]] 
    
def skewness(x):
    
    if len(x) == 0:
        return [0.0, 0.0]

    elif len(x) == 1:
        return [0.0, 0.0]
    
    else:
        return list(sp.stats.skew(x,0))
    
def loglk_norm( x, mu, cov ):
    
    var = multivariate_normal(mean=mu, cov=[[cov[0], cov[2]], [cov[2],cov[1]]])
    return sum( var.logpdf(x) )

def likelihood_ratio_test(llmin, llmax, df):
    return sp.stats.chisqprob(-2.0*(llmin-llmax), df)


def bid_ask_spread(all_dta_minu, tmp_idx):
    x_a = all_dta_minu[tmp_idx][0]
    x_b = all_dta_minu[tmp_idx][1]
    
    #if ask side is empty --- find last ask side and use it
    if (market_depth_a_volume(x_a)==0):
        return abs(find_last_ask_price(all_dta_minu, tmp_idx)-x_b[0][0])
    
    #if bid side is empty -- find last bid side and use it
    if (market_depth_b_volume(x_b)==0):
        return abs(x_a[0][0] - find_last_bid_price(all_dta_minu, tmp_idx))
    
    #calucate difference -- spread
    return abs(x_a[0][0]-x_b[0][0])

def bid_ask_spread_weighted(all_dta_minu, tmp_idx):
    x_a = all_dta_minu[tmp_idx][0]
    x_b = all_dta_minu[tmp_idx][1]
    
    #either bid or ask side is empty -- call just bid_ask_spread function
    if ((market_depth_b_volume(x_b)==0)|(market_depth_a_volume(x_a)==0)):
        return bid_ask_spread(all_dta_minu, tmp_idx)
    
    
    # calculate avg bid on first 10 % of orders
    idx = np.shape(x_b)[0]/10
    
    if (idx==0): #smaller than 10
        idx = np.shape(x_b)[0]
    
    #cumulative price of 10% of bid volume
    cum_bid = 0.0
    for i in range ( idx ):
        cum_bid+=x_b[i][0]
    cum_bid = cum_bid/idx
    
    # calculate avg ask on first 10 % of orders
    idx = np.shape(x_a)[0]/10
    
    if (idx==0): #smaller than 10
        idx = np.shape(x_a)[0]
    
    #cumulative price of 10% of bid volume
    cum_ask = 0.0
    for i in range ( idx ):
        cum_ask+=x_a[i][0]
    cum_ask = cum_ask/idx
    
    return abs(cum_ask-cum_bid)

def market_depth_a_volume(x_a):
    #number of orders
    return (np.shape(x_a)[0])

def market_depth_b_volume(x_b):
    #number of orders
    return (np.shape(x_b)[0])

def market_depth_a_btc(x_a):
    #sum of btc in ask side
    btc_sum = 0.0
    for i in range ( np.shape(x_a)[0] ):
        btc_sum+=x_a[i][1]
    return btc_sum

def market_depth_b_btc(x_b):
    #sum of btc in bid side
    btc_sum = 0.0
    for i in range ( np.shape(x_b)[0] ):
        btc_sum+=x_b[i][1]
    return btc_sum

def find_last_bid_price(all_dta_minu, idx):
    #search for last available bid
    tmp_b = all_dta_minu[idx][1]
    while(market_depth_b_volume(tmp_b)==0):
        idx=idx-1;
        tmp_b = all_dta_minu[idx][1]
        
    return tmp_b[0][0]

def find_last_ask_price(all_dta_minu, idx):
    #search for last available ask
    tmp_a = all_dta_minu[idx][0]
    while(market_depth_a_volume(tmp_a)==0):
        idx=idx-1;
        tmp_a = all_dta_minu[idx][0]
        
    return tmp_a[0][0]

def bid_ask_slope(all_dta_minu, tmp_idx):
    #calculated the volume in the tail that belongs closest to the current price
    #essentially sum until some delta price -- is estimated from data -> from first 10 % of orders
    x_a = all_dta_minu[tmp_idx][0]
    x_b = all_dta_minu[tmp_idx][1]
    
    if (market_depth_b_volume(x_b)==0): #bid is empty
        cum_bid = 0.0
        idx = np.shape(x_a)[0]/10 #use 10% on ask sid
        delta = abs(x_a[idx][0] - x_a[0][0]) #critical value for summation
    else:
        #find delta valule for price for the first 10% of orders on bid side and use it also for ask side
        idx = np.shape(x_b)[0]/10
        delta = abs(x_b[idx][0] - x_b[0][0]) #critical value for summation
    
        #cumulative volume of orders on bid side until the delta price
        cum_bid = 0.0
        for i in range ( idx ):
            cum_bid+=x_b[i][1]
        
    if (market_depth_a_volume(x_a)==0): #ask is empty
        cum_ask = 0
    else:    
        #cumulative volume of orders on ask side until the delta price
        cum_ask = 0.0
        for i in range ( np.shape(x_a)[0] ):
            if ( x_a[i][0]  <= (delta+x_a[0][0]) ):
                cum_ask+=x_a[i][1]
    
    
    return cum_bid, cum_ask
    

def orderbook_stat_features(all_dta_minu, tmp_idx):
    tmp_a = all_dta_minu[tmp_idx][0]
    tmp_b = all_dta_minu[tmp_idx][1]
 
    f = []
    f.append(bid_ask_spread(all_dta_minu, tmp_idx))
    f.append(bid_ask_spread_weighted(all_dta_minu, tmp_idx))
    f.append(market_depth_a_volume(tmp_a))
    f.append(market_depth_b_volume(tmp_b))
    f.append(market_depth_a_volume(tmp_a)-market_depth_b_volume(tmp_b))
    f.append(market_depth_a_btc(tmp_a))
    f.append(market_depth_b_btc(tmp_b))
    f.append(market_depth_a_btc(tmp_a)-market_depth_b_btc(tmp_b))
    cum_bid, cum_ask = bid_ask_slope(all_dta_minu, tmp_idx)
    f.append(cum_bid)
    f.append(cum_ask)
    return f




 

In [7]:
# --- extract features w.r.t. minute ---
features_minu = [] 
# ask mean price, mean amount, var price, var amount, 
# bid mean price, mean amount, var price, var amount,
# ask skew price, skew amount
# bid skew price, skew amount, 
# ask request, bid request 

for i in range( len(all_dta_minu) ):
    tmp_a = all_dta_minu[i][0]
    tmp_b = all_dta_minu[i][1]
#   shape abov e: # by [price, amount]
    
#   mle_norm_2d() replaced by Nino func  
    tmpft = mle_norm_2d(tmp_a)
    tmp = tmpft[0] + tmpft[1]
    
#   mle_norm_2d() replaced by Nino func  
    tmpft = mle_norm_2d(tmp_b)
    tmp += tmpft[0]
    tmp += tmpft[1]
    
    #   skewness feature
    tmp += skewness(tmp_a)
    tmp += skewness(tmp_b)
    
    # financial-like features -- spread, slope, depth
    tmp = orderbook_stat_features(all_dta_minu,i)
    
    features_minu.append( tmp )
    
    # amount of requests
    tmp += [len(all_dta_minu[i][0]), len(all_dta_minu[i][1])] 
    
#     if len(all_dta_minu[i][0]) ==0 or len(all_dta_minu[i][1])==0:
#             print len(all_dta_minu[i][0]), len(all_dta_minu[i][1])
            
#  # of miniutes by # features
features_minu = np.reshape( features_minu, (len(features_minu), -1) )

print np.shape( np.asarray(features_minu) )

(539157, 12)


In [7]:
def training_testing_garch(features_minu, vol_hour, all_loc_hour, \
                                  order_minu, order_hour, train_split_ratio, price_minu):
    tmpcnt = len(vol_hour)
    tmp_split = int(train_split_ratio*(tmpcnt - order_hour - 1)) + order_hour
    
    minu_idx = all_loc_hour[tmp_split]
    
    price_train = price_minu[:minu_idx+1]
    price_test  = price_minu[minu_idx+1:]
    
    return_train = []
    for i in range(1, len(price_train)):
        return_train.append( (price_train[i]-price_train[i-1])/(price_train[i-1]+1e-5)*100 )
        
    return_test = []
    for i in range(1, len(price_test)):
        return_test.append( (price_test[i]-price_test[i-1])/(price_test[i-1]+1e-5)*100 )
            
    # vol train, return train, vol test, return test
    return vol_hour[1:tmp_split+1], return_train, vol_hour[tmp_split+1:], return_test

In [10]:
# --- obtain training and testing data, garch ---
para_order_minu = 10
para_order_hour = 16
para_train_split_ratio = 0.8
file_postfix = "garch" 
#mle_norm, map_norm, map_lognorm 

vol_train, rt_train, vol_test, rt_test = training_testing_garch(features_minu, rvol_hour, all_loc_hour, \
                                   para_order_minu, para_order_hour, para_train_split_ratio, price_minu)

print np.shape(vol_train), np.shape(rt_train), np.shape(vol_test), np.shape(rt_test)
print len(rt_train)+len(rt_test), len(vol_train)+len(vol_test)

np.asarray(vol_train).dump("../dataset/bitcoin/training_data/voltrain_"+file_postfix+".dat")
np.asarray(rt_train).dump("../dataset/bitcoin/training_data/rttrain_"  +file_postfix+".dat")
np.asarray(vol_test).dump("../dataset/bitcoin/training_data/voltest_"  +file_postfix+".dat")
np.asarray(rt_test).dump("../dataset/bitcoin/training_data/rttest_"    +file_postfix+".dat")

(8515,) (419602,) (2125,) (119553,)
539155 10640


In [18]:
# --- obtain training and testing data, arima, structural time series ---
para_order_minu = 10
para_order_hour = 16
para_train_split_ratio = 0.8
file_postfix = "stat" 
#mle_norm, map_norm, map_lognorm 

xtrain, extrain, xtest, extest = training_testing_statistic(features_minu, rvol_hour, all_loc_hour, \
                                   para_order_minu, para_order_hour, para_train_split_ratio)

print np.shape(xtrain), np.shape(extrain), np.shape(xtest), np.shape(extest)

np.asarray(xtrain).dump("../dataset/bitcoin/training_data/xtrain_"+file_postfix+".dat")
np.asarray(xtest ).dump("../dataset/bitcoin/training_data/xtest_" +file_postfix+".dat")
np.asarray(extrain).dump("../dataset/bitcoin/training_data/extrain_"+file_postfix+".dat")
np.asarray(extest ).dump("../dataset/bitcoin/training_data/extest_" +file_postfix+".dat")


(8515,) (8515, 140) (2125,) (2125, 140)


In [15]:
tmpres= np.diff(ytest)
print sqrt(mean(tmpres*tmpres)) 


0.0555666200111


In [11]:

# --- obtain training and testing data, plain regression ---
para_order_minu = 10
para_order_hour = 16
para_train_split_ratio = 0.8
file_postfix = "norm_v_minu_reg" 
#mle_norm, map_norm, map_lognorm 

# features_minu, req_minu, pvol_hour, all_loc_hour
x, y = prepare_feature_target( features_minu, [], rvol_hour, all_loc_hour, \
                                                        para_order_minu, para_order_hour )
print len(x)
    
xtrain, ytrain, xtest, ytest = training_testing_plain_regression(x, y, para_train_split_ratio)    
print np.shape(xtrain), np.shape(ytrain), np.shape(xtest), np.shape(ytest)

# np.asarray(xtrain).dump("../dataset/bitcoin/training_data/xtrain_"+file_postfix+".dat")
# np.asarray(xtest ).dump("../dataset/bitcoin/training_data/xtest_" +file_postfix+".dat")
# np.asarray(ytrain).dump("../dataset/bitcoin/training_data/ytrain_"+file_postfix+".dat")
# np.asarray(ytest ).dump("../dataset/bitcoin/training_data/ytest_" +file_postfix+".dat")


10625
(8500, 156) (8500,) (2125, 156) (2125,)


In [16]:
# --- obtain training and testing data, mixture: linear, mlp --- 
para_order_minu = 10
para_order_hour = 16
para_train_split_ratio = 0.8 
file_postfix = "norm_v_minu_mix"

# features_minu, req_minu, pvol_hour, all_loc_hour
x, y = prepare_feature_target( features_minu, [], rvol_hour, all_loc_hour, \
                                                        para_order_minu, para_order_hour )
print len(x)

xtrain, ytrain, xtest, ytest = training_testing_mixture_mlp(x, y, para_train_split_ratio)

print len(xtrain[0]), np.shape(ytrain), np.shape(ytest)

np.asarray(xtrain).dump("../dataset/bitcoin/training_data/xtrain_"+file_postfix+".dat")
np.asarray(xtest ).dump("../dataset/bitcoin/training_data/xtest_" +file_postfix+".dat")
np.asarray(ytrain).dump("../dataset/bitcoin/training_data/ytrain_"+file_postfix+".dat")
np.asarray(ytest ).dump("../dataset/bitcoin/training_data/ytest_" +file_postfix+".dat")

#  done for training-test preparaion 

10625
2 (8500,) (2125,)


In [28]:
# --- obtain training and testing data, mixture: rnn --- 
para_order_minu = 10
para_order_hour = 64
para_train_split_ratio = 0.8
file_postfix = "neu_norm_v_minu_mix"

# features_minu, req_minu, pvol_hour, all_loc_hour
x, y = prepare_feature_target( features_minu, [], pvol_hour, all_loc_hour, \
                                                        para_order_minu, para_order_hour )
print len(x)

xtrain, ytrain, xtest, ytest = training_testing_mixture_rnn(x, y, para_train_split_ratio)

print len(xtrain[0]), np.shape(ytrain), np.shape(ytest)

np.asarray(xtrain).dump("../dataset/bitcoin/training_data/xtrain_"+file_postfix+".dat")
np.asarray(xtest ).dump("../dataset/bitcoin/training_data/xtest_" +file_postfix+".dat")
np.asarray(ytrain).dump("../dataset/bitcoin/training_data/ytrain_"+file_postfix+".dat")
np.asarray(ytest ).dump("../dataset/bitcoin/training_data/ytest_" +file_postfix+".dat")


10577
2 (8461,) (2116,)


In [ ]:
# --- likelihood evaluation ---
from scipy.stats import multivariate_normal

n_sample = 100
tmpidx = range(len(all_dta_minu))
np.random.shuffle(tmpidx)
sample_dta_minu = all_dta_minu[tmpidx[:n_sample]]

for i in sample_dta_minu:
    tmpa = i[0]
    tmpb = i[1]
    
    tmp_para  = map_norm_2d(tmpa)
    loglk_min = loglk_norm( tmpa, tmp_para[0], tmp_para[1] )
    print tmp_para[1][2]
#     tmp_rv = sp.stats.norm( tmp_para[0][0], tmp_para[1][0])
#     print sp.stats.kstest( [i[0] for i in tmpa] ,tmp_rv.cdf )
    
#     tmp_rv = sp.stats.norm( tmp_para[0][1], tmp_para[1][1])
#     print sp.stats.kstest( [i[1] for i in tmpa] ,tmp_rv.cdf )

#     tmp_para = map_norm_2d(tmpa)
#     print sum(loglk_norm( tmpa, tmp_para[0], tmp_para[1] ))
    
    tmplog = [[log(i[0]+1e-07), log(i[1]+1e-07)] for i in tmpa]
    tmp_para  = mle_norm_2d( tmplog )
    print tmp_para[1][2]
    
#     loglk_max = loglk_norm( tmplog, tmp_para[0], tmp_para[1] )
    
#     tmp_rv = sp.stats.norm( tmp_para[0][0], tmp_para[1][0] )
#     print sp.stats.kstest( [i[0] for i in tmplog] ,tmp_rv.cdf )
    
#     tmp_rv = sp.stats.lognorm( tmp_para[0][1], tmp_para[1][1] )
#     print sp.stats.kstest( [i[1] for i in tmplog] ,tmp_rv.cdf )

#     print loglk_max,loglk_min, likelihood_ratio_test(loglk_min, loglk_max, 1)
